In [1]:
import numpy as np
import cv2
import pandas as pd

In [2]:
# help(cv2.threshold)

Help on built-in function threshold:

threshold(...)
    threshold(src, thresh, maxval, type[, dst]) -> retval, dst
    .   @brief Applies a fixed-level threshold to each array element.
    .   
    .   The function applies fixed-level thresholding to a multiple-channel array. The function is typically
    .   used to get a bi-level (binary) image out of a grayscale image ( #compare could be also used for
    .   this purpose) or for removing a noise, that is, filtering out pixels with too small or too large
    .   values. There are several types of thresholding supported by the function. They are determined by
    .   type parameter.
    .   
    .   Also, the special values #THRESH_OTSU or #THRESH_TRIANGLE may be combined with one of the
    .   above values. In these cases, the function determines the optimal threshold value using the Otsu's
    .   or Triangle algorithm and uses it instead of the specified thresh.
    .   
    .   @note Currently, the Otsu's and Triangle methods a

In [3]:
#Read the image
base = './'
im = cv2.imread(base + 'skyscraper.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
#ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
winSize = 7
blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
edges = cv2.Canny(blured,100,220,apertureSize = 7, L2gradient=True)



#cv2.imshow('houghlines',im)
cv2.imwrite(base+'gray.jpg', gray)
cv2.imwrite(base+'equ.jpg', equ)
cv2.imwrite(base+'blured.jpg', blured)
cv2.imwrite(base+'edges.jpg', edges)

#cv2.imwrite(base+'thresh.jpg', thresh)

#A few changes to check git functionality

True

In [ ]:
# help(cv2.Canny)

In [ ]:
# help(cv2.HoughLines)

In [4]:
im.shape

(757, 543, 3)

In [5]:
height = im.shape[0]
width = im.shape[1]

In [6]:
def draw_line (img, rho, theta, color=(0,255,0)):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(np.around(x0 + 1000*(-b)))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
    y1 = int(np.around(y0 + 1000*(a)))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
    x2 = int(np.around(x0 - 1000*(-b)))   # But we need integers, so use int() function after that, ie int(np.around(x))
    y2 = int(np.around(y0 - 1000*(a)))
    cv2.line(img,(x1,y1),(x2,y2),color,2)
    print (rho, theta, color)

In [7]:
np.pi / 2

1.5707963267948966

In [8]:
img = im.copy()
rho_minus = 0
rho_plus = np.inf

rho_hor_max = 0
hor_max_found = False
rho_hor_min = im.shape[0]
hor_min_found = False
eps = 0.01
lines = cv2.HoughLines(edges,1,np.pi/180, int (min(im.shape[0], im.shape[1]) / 3))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
                hor_max_found = True
            if (rho < rho_hor_min):
                rho_hor_min = rho
                hor_min_found = True
        #draw_line (img, rho, theta)
        
red = (0, 0, 255)
draw_line (img, rho_minus, theta_minus, red)
draw_line (img, rho_plus, theta_plus, red)

if hor_min_found:
    draw_line (img, rho_hor_min, np.pi / 2, red)
if hor_max_found:
    draw_line (img, rho_hor_max, np.pi / 2, red)
cv2.imwrite(base+'houghlines_red.jpg', img)
#cv2.imshow('houghlines',img)

-341.0 2.8972466 (0, 0, 255)
190.0 0.2617994 (0, 0, 255)


True

In [ ]:
# img = im.copy()
# lines = cv2.HoughLinesP(edges,1,np.pi/180,80, minLineLength = 100, maxLineGap = 10)
# for line in lines:
#     for x1,y1,x2,y2 in line:
#         cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
# cv2.imwrite(base+'houghlines_p.jpg', img)
# cv2.imshow('houghlines',img)
# cv2.waitKey()


In [9]:
# Given y, this function returns x from (rho, theta) line equation
def x_from_y(rho, theta, y):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 
    
    return ((rho / cos_t) - ((y * sin_t) / cos_t))

In [10]:
# Given x, this function returns y from (rho, theta) line equation
def y_from_x(rho, theta, x):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 
    
    return (-((x * cos_t) / sin_t) + (rho / sin_t)) 

In [11]:
# Given a point and the rectangle, this function checks 
# if this point lies inside the rectangle
def check_point(x, y, left_x, right_x, up_y, down_y):
    if (left_x <= x) and (x <= right_x) and (up_y <= y) and (y <= down_y):
        return (int(np.around(x)), int(np.around(y)))
    else:
        return None

In [12]:
# A line which traverses some rectangle has two points of traversal.
#
# This function finds four possible traversal points and drops those of them 
# which are not inside the rectangle
def get_traversal_points(rho, theta, left_x, right_x, up_y, down_y):      
    
    #Find four possible points of traversal
    up = (x_from_y(rho, theta, up_y), up_y)
    right = (right_x, y_from_x(rho, theta, right_x))
    down = (x_from_y(rho, theta, down_y), down_y)
    left = (left_x, y_from_x(rho, theta, left_x))
    
    #If point is inside, we add it to the list; otherwise we add None
    intersect_points = list(map(lambda p: check_point(p[0], p[1], left_x, right_x, up_y, down_y), [up, right, down, left]))
    
    #Drop all None values
    intersect_points = [point for point in intersect_points if point != None]
    
    #Check if exactly two points were found
    if len(intersect_points) != 2:
        print('Error! len(intersect_points) != 2, intersect_points = {}'.format(intersect_points))
        return None
    else:
        return sorted(intersect_points, key=lambda x: x[0])  

In [13]:
print(get_traversal_points(rho_minus, theta_minus, 0, width, 0, height))
print(get_traversal_points(rho_plus, theta_plus, 0, width, 0, height))

[(351, 0), (540, 757)]
[(0, 734), (197, 0)]


In [14]:
# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid
down_plus, up_plus = get_traversal_points(rho_plus, theta_plus, 0, width, 0, height)
up_minus, down_minus = get_traversal_points(rho_minus, theta_minus, 0, width, 0, height)

crop_height = min(down_minus[1], down_plus[1])
print(crop_height)

734


In [19]:
strip = im[:, up_plus[0]:up_minus[0]]
strip_edges = edges[:, up_plus[0]:up_minus[0]]
cv2.imwrite(base + 'strip.jpg', strip)

True

In [20]:
strip_img = strip.copy()
rho_minus = 0
rho_plus = np.inf

rho_hor_max = 0
hor_max_found = False
rho_hor_min = strip.shape[0]
hor_min_found = False
eps = 0.01
lines = cv2.HoughLines(strip_edges,1,np.pi/180, int (min(strip.shape[0], strip.shape[1]) / 3))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
                hor_max_found = True
            if (rho < rho_hor_min):
                rho_hor_min = rho
                hor_min_found = True
        #draw_line (img, rho, theta)
        
red = (0, 0, 255)
draw_line (strip_img, rho_minus, theta_minus, red)
draw_line (strip_img, rho_plus, theta_plus, red)

if hor_min_found:
    draw_line (strip_img, rho_hor_min, np.pi / 2, red)
if hor_max_found:
    draw_line (strip_img, rho_hor_max, np.pi / 2, red)
cv2.imwrite(base+'strip_houghlines_red.jpg', strip_img)
#cv2.imshow('houghlines',img)

431.0 2.3561945 (0, 0, 255)
0.0 0.0 (0, 0, 255)
72.0 1.5707963267948966 (0, 0, 255)
78.0 1.5707963267948966 (0, 0, 255)


True

In [ ]:
crop = im[0:crop_height, :]
cv2.imwrite(base+'cropped.jpg', crop)

In [ ]:
# Get the points of traversal for the cropped picture
crop_down_plus, crop_up_plus = get_traversal_points(rho_plus, theta_plus, 0, width, 0, crop_height)
crop_up_minus, crop_down_minus = get_traversal_points(rho_minus, theta_minus, 0, width, 0, crop_height)

In [ ]:
# Create a transform and apply it
from_points = np.float32([crop_down_plus, crop_up_plus, crop_up_minus, crop_down_minus])
to_points = np.float32([(0, crop_height), (0, 0), (width, 0), (width, crop_height)])

M = cv2.getPerspectiveTransform(from_points, to_points)
dst = cv2.warpPerspective(crop, M, (width, crop_height))

cv2.imwrite(base+'transformed.jpg', dst)